In [1]:
from pyspark import SparkConf, SparkContext
# SparkConf : 환경 설정
# SparkContext : 실제 개발 프로그램. 드라이브
#conf = SparkConf().setMaster("yarn") # yarn으로 하면 자동으로 하둡을 사용하게 된다
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf) # sc 변수명 바꾸면 안된다. 실무에서는 스파크 서브밋 이라는 것을 사용한다. 그때 필수

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/22 14:35:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Reduce
- 사용자가 지정하는 함수를 받아(task) 여러 개의 값을 하나로 줄여준다.
- 파티션 별로 작업이 일어난다.

In [3]:
from operator import add

In [4]:
add(1,3)

4

In [5]:
sample_rdd = sc.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [6]:
sample_rdd.reduce(add)

15

In [10]:
# 파티션 옵션을 따로 설정하지 않으면 기본적으로 파티션이 1개
sample_rdd_p1 = sc.parallelize([1,2,3,4])
sample_rdd_p1.glom().collect() # glom은 파티션으로 나뉘어진 정보를 보여준다

[[1, 2, 3, 4]]

In [11]:
sample_rdd_p2 = sc.parallelize([1,2,3,4],2) #파티션 2개 설정
sample_rdd_p2.glom().collect() 

[[1, 2], [3, 4]]

In [12]:
# 파티션이 1개인 상태에서 reduce
sample_rdd_p1.reduce(lambda x, y : (x*2)+y)

26

In [13]:
# 파티션이 2개인 상태에서 reduce
sample_rdd_p2.reduce(lambda x, y : (x*2)+y)

18

In [17]:
# 파티션이 3개인 상태에서 reduce
sample_rdd_p3 = sc.parallelize([1,2,3,4],3) #파티션 3개 설정
sample_rdd_p3.reduce(lambda x, y : (x*2)+y)

18

In [18]:
sample_rdd_p3.glom().collect() 

[[1], [2], [3, 4]]

In [16]:
# 데이터가 4개인데 3개의 파티션으로 구성하면 앞쪽꺼는 하나씩, 뒤에는 다 몰려서 들어간다

In [19]:
# 파티션 4개로 하면 1개로 했을 때와 같은 결과가 나온다

동일한 연산을 수행하는데, 파티션의 개수에 따라서 연산의 결과가 달라진다. 연산자의 우선순위를 항상 주의해야 한다.

# Fold
- `reduce`와 비슷하지만, `zeroValue`에 넣어 놓고 싶은 시작값을 지정해서 `reduce`
- `zeroValue`는 파티션 마다 계산이 일어날 때 하나씩 더해지는 값

In [22]:
sample_rdd = sc.parallelize([2,3,4],4)
sample_rdd.glom().collect()

[[], [2], [3], [4]]

In [23]:
sample_rdd.reduce(lambda x, y : (x*y))

24

In [32]:
sample_rdd.fold(1, lambda x, y : (x*y)) # [1,1], [1,2],[1,3],[1,4] 매개변수가 2개니까 1(설정값, 시작값)이 추가된다.
# 각 데이터의 파티션별로 우선 연산이 이루어 진다
# (1*1)*(1*2)*(1*3)*(1*4)

24

In [33]:
sample_rdd.fold(2, lambda x, y : (x*y)) # [2,2], [2,2],[2,3],[2,4] 매개변수가 2개니까 1(설정값, 시작값)이 추가된다.
# 각 데이터의 파티션별로 우선 연산이 이루어 진다
# (2*2)*(2*2)*(2*3)*(2*4)

768

In [34]:
# 설정값으로 연산에 필요한 구조를 맞춰주고 빈값이 있으면 채운다

# groupBy
- 그루핑할 기준 함수를 받아서 reduction

In [35]:
sample_rdd = sc.parallelize([1,1,2,3,5,6,7,10,3])

In [38]:
result = sample_rdd.groupBy(lambda x : x%2).collect()
result

[(1, <pyspark.resultiterable.ResultIterable at 0x7f2f2019a580>),
 (0, <pyspark.resultiterable.ResultIterable at 0x7f2f2019aac0>)]

In [40]:
for k, v in result:
    print(k, list(v))

1 [1, 1, 3, 5, 7, 3]
0 [2, 6, 10]


In [41]:
sorted([(x, sorted(y)) for (x, y) in result]) # sorted : 내부 Iterable 객체를 정렬한 후 리스트로 리턴

[(0, [2, 6, 10]), (1, [1, 1, 3, 3, 5, 7])]

# Aggregate
- `RDD.aggregate(zeroValue, seqOp, combOp)`
    - `zeroValue` : 각 파티션에서 누적할 시작 값
    - `seqOp` : 타입 변경 함수
        - 파티션 내에서 벌어지는 연산을 담당
    - `combOp` : 모든 결과를 하나로 합쳐주는 역할
        - 파티션 밖에서, 집계해 주는 연산
    - 파티션 단위의 연산 결과를 합쳐주는 과정을 거치게 된다.

In [43]:
sample_rdd = sc.parallelize([1,2,3,4],2)
sample_rdd.glom().collect()

[[1, 2], [3, 4]]

In [46]:
seqOp = lambda x, y : (x[0]+y, x[1]+1) # 파티션 내의 연산
combOp = lambda x, y : (x[0]+y[0], x[1]+y[1]) # 파티션의 모든 결과를 최종 연산

In [47]:
sample_rdd.aggregate((0,0), seqOp, combOp)

(10, 4)

In [ ]:
(0,1) (2,1) (4,1)
(2,2)
(10,7)